# Content-based Filtering

Aproach:
- Table mit Features und Ratings erstellen, ohne User - auf Basis des OMDB Datasets

- sklearn-kNN mit cosine-similarity darauf anwenden

- Funktion schreiben, die auf basis des ratings der neighbours das rating eines Filmes vorhersagt.

## This is a WIP: Decided to do omdb data cleaning first - bevore a useful model can be applied

Sources:

#### https://heartbeat.fritz.ai/recommender-systems-with-python-part-i-content-based-filtering-5df4940bd831

#### https://www.kaggle.com/johnwill225/movie-recommendations

#### https://towardsdatascience.com/how-we-built-a-content-based-filtering-recommender-system-for-music-with-python-c6c3b1020332


In [215]:
import pandas as pd
import numpy as np
import string
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA

In [216]:
movies = pd.read_csv('../data/preprocessed/movies_id_updated.csv', sep=',')
average_ratings = pd.read_csv('../data/preprocessed/average_ratings.csv')
omdb = pd.read_csv('../data/preprocessed/omdb_relevant.csv')
ratings = pd.read_csv('../data/raw/ratings.csv')

In [217]:
average_ratings.head()

,Unnamed: 0,movieID,title,imdbID,ratingAvg,imdbRating Avg,Rotten TomatoesAvg,Internet Movie Database Avg,Metascore Avg,Metacritic Avg
0,844798,6990.0,The Great Train Robbery,tt0000439,3.696970,3.65,5.00,3.65,NaN,NaN
1,850743,7065.0,The Birth of a Nation,tt0004972,3.228261,3.20,4.65,3.20,NaN,NaN
2,859392,7243.0,Intolerance: Love's Struggle Throughout the Ages,tt0006864,3.928571,3.90,4.85,3.90,4.95,4.95
3,886762,8511.0,The Immigrant,tt0008133,3.750000,3.85,NaN,3.85,NaN,NaN
4,971742,48374.0,Otets Sergiy,tt0008395,3.000000,3.55,NaN,3.55,NaN,NaN


In [218]:
omdb.head()

,Year,Rated,Released,Runtime,imdbRating,imdbVotes,imdbID,Rotten Tomatoes,Metacritic,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,1995.0,0.0,11.0,81.0,8.3,852896.0,tt0114709,10.0,9.5,NaN,3.0,NaN,NaN
1,1995.0,1.0,12.0,104.0,7.0,294340.0,tt0113497,5.4,3.9,NaN,NaN,NaN,NaN
2,1993.0,2.0,12.0,103.0,7.0,41401.0,tt0107050,6.3,5.3,NaN,NaN,NaN,NaN
3,1995.0,3.0,12.0,124.0,5.9,9222.0,tt0114885,5.6,NaN,NaN,NaN,NaN,NaN
4,1995.0,1.0,12.0,106.0,6.0,33005.0,tt0113041,4.8,4.9,NaN,NaN,NaN,1.0


In [219]:
omdb['Oscars_won'] = omdb['Oscars_won'].replace(r'NaN', '0').astype(float)
omdb['Oscars_nominated'] = omdb['Oscars_nominated'].replace(r'NaN', '0').astype(float)
omdb['Golden_globe_won'] = omdb['Golden_globe_won'].replace(r'NaN', '0').astype(float)
omdb['Golden_globe_nominated'] = omdb['Golden_globe_nominated'].replace(r'NaN', '0').astype(float)

In [231]:
omdb.isna().any()

Year                      False
Rated                      True
Released                   True
Runtime                    True
imdbRating                 True
imdbVotes                  True
imdbID                    False
Rotten Tomatoes            True
Metacritic                 True
Oscars_won                 True
Oscars_nominated           True
Golden_globe_won           True
Golden_globe_nominated     True
dtype: bool

In [220]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

In [221]:
#Shuffling and then restricting data here to save runtime -> TODO: Find better/faster solution
ratings = ratings.sample(axis=0,frac=1)
restricted_ratings = ratings.iloc[:10192]

In [222]:
y = np.array(restricted_ratings['rating'])

In [223]:
X = np.array(omdb)

In [224]:
y.shape

(10192,)

In [225]:
X.shape

(10192, 13)

AttributeError: 'numpy.ndarray' object has no attribute 'isna'

In [226]:
le = preprocessing.LabelEncoder()
for i in range(0,12):
    X[:,i] = le.fit_transform(X[:,i])

In [227]:
y = le.fit_transform(y)

In [228]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/5)

In [229]:
print("Training")
knn.fit(X_train, y_train)

Training


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# Do predictions
print("Do Prediction")
pred = knn.predict(X_test)

In [ ]:
# Check Accuracy
print("Accuracy : {}".format(accuracy_score(y_test, pred)))
#RMSE for ratings [0,5]
print("RMSE : {}".format(mean_squared_error(y_test, pred, squared=False)))

In [ ]:
## function that predicts the rating of a movie from its imdbID and its 20 nearest neighbors

def predict_movie_rating(imdbID, userID):
    # calculate the average rating of each movie
    # get 20 (or x) nearest neighbors that were rated by userID
    # compute rating of movie(imbdID) based on the rating of the 20 nearest neighbours
        #Make sure not to take NaNs into account
    # return rating prediciton
    